In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
#| default_exp chatbot

In [ ]:
#| export

import pandas as pd
import sys
import os
from pathlib import Path
from fastcore.all import patch
import matplotlib.pyplot as plt
import seaborn as sns

# Load data / Manually input the data

In [ ]:
#| export

# With this I deal with notebooks and .py files to get the parent folder of the running file
# since __file__ is not defined in notebooks
def get_base_dir():
    try:
        # Works in .py scripts
        return Path(__file__).resolve().parent
    except NameError:
        # Fallback for Jupyter notebooks
        return Path(os.getcwd()).resolve()

In [ ]:
base_dir = get_base_dir()
path_to_data = base_dir.parent / "resources"
file_name = "financial_data_chatbot.csv"
file_path = path_to_data / file_name


# Data class for the chatbot

## Scheme for the chatbot:

In [ ]:
#| export
class financial_chatbot():

    def __init__(self, file_path: Path):
        
        self._data = pd.read_csv(file_path)
        
        self._last_query: str = None
        self._current_query:str = None

        self._available_companies = self._data['Company'].unique().tolist()
        self._current_companies = []

        self._last_action = []
        self._current_action = []

        # Financial facts in data
        self._available_fin_facts = self._data.columns.difference(['Company', 'Year']).to_list()
        self._last_fin_facts = []
        self._current_fin_facts = []

        # The dictionary that will hold the final mapping
        self.action_for = {}
        self._keywords_print = set(['print', 'give me', 'show', 'display', 'output'])
        self._keywords_plot = set(['plot', 'graph', 'chart', 'visualize'])


        # Main keywords of the data
        # Words that indicate growth/percentage intent
        self._growth_keywords = {'grow', 'growth', 'percentage', 'percent', '%', 'increase', 'change', 'yoy', 'annualized'}

        # Stopwords to ignore when matching tokens (keeps domain words like 'cash' and 'flow')
        self._stopwords = {'the', 'and', 'of', 'from', 'a', 'an', 'in', 'on', 'at', 'for', 'to', 'total'}


        # Chatbot Hello
        self.print_intro()

        # Initiate some variables
        self.initiate_actions_dictionary()

    def initiate_actions_dictionary(self):
        """Defined below"""
        return

    def get_response(self, message="Your message:"):        
        self._last_query = self._current_query
        self._current_query = input(message)
        print()

        # If user wants to exit, stop the program
        if self._current_query.lower() in ["exit", "quit"]:
            raise SystemExit("Exiting chatbot.")



    def print_intro(self):
        intro_text = """
        Welcome to the Financial Chatbot!
        You can ask me questions about financial data.

        Note: All values, unless stated as percentages, 
        are expressed in millions of dollars (M$). 
        
        Type 'exit' to end the conversation.

        How can I assist you today? 
        """
        print(intro_text)
        


In [ ]:
chatbot = financial_chatbot(file_path=file_path)

In [ ]:
chatbot.print_intro()

In [ ]:
chatbot._available_companies

0. I need a company or companies to query
    + I should have a current company field that gets updated when parsing for companies
    + If there is no Company in the query I continue applyin things to the `current_company`
    + If after parsing everything `current_company` is empty, I have to ask for the company.
    
1. From the query I need an action keyword:
    - actions: 
        + ['show', 'print', 'get', 'display', 'tell', 'give', "how", "what"]
        + ['plot', 'graph', 'chart', 'visualize', 'draw']
2. I also need a subject keyword or keywords to ignore:
    - stopwords = {'the', 'and', 'of', 'from', 'a', 'an', 'in', 'on', 'at', 'for', 'to', 'total'}
    
3. And adjectives keywords to classify:
    - adjectives: 
        + ['growth', 'change', 'increase', 'decrease']


## Methods to parse and prompt for missing information

### Method to parse and update the companies

In [ ]:
#| export
@patch
def parse_companies(self: financial_chatbot):
    comp = []
    for company in self._available_companies:
        if company.lower() in self._current_query.lower():
            comp.append(company)
    
    # if len comp is not zero, update current companies
    if len(comp) > 0:
        self._current_companies = comp

    return comp


In [ ]:
chatbot._current_query = "Give me the Net income for APPLE."
print(f"Parsed companies: {chatbot.parse_companies()}")
print(f"Active companies: {chatbot._current_companies}")

If no companies in the query, ask for what companies

### Method to initiate the possible actions

Associate keywords with a function

In [ ]:
# The list of all trigger words (synonyms)
print_keywords = set(['print', 'give me', 'show', 'display', 'output'])
plot_keywords = set(['plot', 'graph', 'chart', 'visualize'])

# The dictionary that will hold the final mapping
action_for = {}


Function to populate the dictionary that maps keywords with functions

In [ ]:
#| export
def associate_action(keywords, function, dictionary):
    for keyword in keywords:
        dictionary[keyword] = function
    
    return dictionary

Populate and test the dictionary

In [ ]:
action_for = associate_action(print_keywords, print, action_for)
action_for = associate_action(plot_keywords, plt.plot, action_for)

In [ ]:
action_for['show']("Hello, World!")

In [ ]:
action_for['graph']([1, 2, 3], [4, 5, 6])

Include this functionality as a method

In [ ]:
#| export
@patch
def initiate_actions_dictionary(self: financial_chatbot):
    # Map print and plot keywords to their respective functions
    associate_action(self._keywords_print, print, self.action_for)
    associate_action(self._keywords_plot, plt.plot, self.action_for)

### Parse for the action

In [ ]:
#| export
@patch
def parse_action(self: financial_chatbot):
    # Separate the query into words
    query = self._current_query.lower().replace(',', '').replace('.', '').split()
    # Convert to sets for easier matching
    query_set = set(query)

    # Get all possible actions
    actions_set = set(self.action_for.keys())

    # Find common keywords between the query and the action keywords
    queried_actions = actions_set.intersection(query_set)

    # If any actions were found, update the current actions
    if len(queried_actions) > 0:
        self._last_action = self._current_action
        self._current_action = list(queried_actions)

    return queried_actions

In [ ]:
chatbot._current_query = "Please plot and print the revenue for Microsoft."
chatbot.initiate_actions_dictionary()
chatbot.parse_action()

### Parse for the financial facts

Let's use the re module to look for patterns in the query.

In [ ]:
#| export
import re
from typing import List

---

In [ ]:
COLUMNS = [
    'Cash Flow from Operations',
    'Cash Flow from Operations growth (%)',
    'Net Income',
    'Net Income growth (%)',
    'Total Assets',
    'Total Assets growth (%)',
    'Total Liabilities',
    'Total Liabilities growth (%)',
    'Total Revenue',
    'Total Revenue growth (%)'
]

# Words that indicate growth/percentage intent
GROWTH_KEYWORDS = {'growth', 'percentage', 'percent', '%', 'increase', 'change', 'yoy', 'annualized'}

# Stopwords to ignore when matching tokens (keeps domain words like 'cash' and 'flow')
STOPWORDS = {'the', 'and', 'of', 'from', 'a', 'an', 'in', 'on', 'at', 'for', 'to', 'total'}



In [ ]:
#| export

# will extract all the words from a string. Notice how it ignores punctuation like parentheses and %
# re.findall(r"\w+", "Total Liabilities growth (%)")
# → ['Total', 'Liabilities', 'growth']
# compiling the regex once so it can be reused efficiently
_word_re = re.compile(r"\w+")

# Tokenizer that obtains all the words in a text
def _tokens(text: str) -> List[str]:
    return _word_re.findall(text.lower())



In [ ]:
#| export

def preprocess_columns(columns, stopwords):
    """
    Converts a list of column names into metadata dictionaries
    containing:
      - original name
      - lowercase token set (without stopwords)
      - is_growth flag
    """
    cols_meta = []
    for col in columns:
        col_lower = col.lower()
        # Flag columns that refer to growth or percentages
        is_growth_col = 'growth' in col_lower or '(%' in col_lower or '%' in col_lower

        # Remove 'growth' and punctuation for base token extraction
        base_text = (
            col_lower.replace('growth', '')
            .replace('(%', '')
            .replace(')', '')
            .replace('%', '')
        )

        # Tokenize and remove stopwords
        col_tokens = [t for t in _tokens(base_text) if t not in stopwords]

        cols_meta.append({
            'name': col,
            'tokens': set(col_tokens),
            'is_growth': is_growth_col
        })
    return cols_meta

In [ ]:
cols_meta = preprocess_columns(columns=COLUMNS, stopwords=STOPWORDS)
display(cols_meta)

In [ ]:
#| export

# Prefer columns that match the growth intent; but if none produce positive score, allow the other group.
def best_match(part_tokens, cols_meta, allow_growth_flag):
    """ Return the column/fact that best matches the query 
        best (meta dictionary)
        score (int)
    """
    best = None
    best_score = 0
    
    # I scan each column (All my financial facts)
    for meta in cols_meta:

        # I check if my query and the current fact have different growth flag (Then I go to the next column/financ fact)
        if meta['is_growth'] != allow_growth_flag:
            continue

        # how many tokens are the same in my query and the financial fact?
        score = len(part_tokens & meta['tokens'])

        # If the query contains the fact give it more points (Special cases)
        if ( (meta['name'].lower() in ' '.join(part_tokens)) ):
            score += 1

        # Throughout my scan, I save the financial fact with the best score
        if score > best_score:
            best = meta
            best_score = score
            
    return best, best_score



In [ ]:
query = "Find me the revenue and perc income"
print(_tokens(query))


parts = [p.strip() for p in re.split(r',|\band\b', query.lower()) if p.strip()]
print(parts)

for part in parts:
    part_tokens = set(t for t in _tokens(part) if t not in STOPWORDS)
    print("part: ", part)
    print("\t part_tokens:", part_tokens)

print(10*"-")

part_tokens = set(t for t in _tokens(parts[0]) if t not in STOPWORDS)
# bm = best_match(part_tokens, cols_meta, allow_growth_flag=True)
bm = best_match(part_tokens, cols_meta, allow_growth_flag=False)

print("Best_match between my columns and:")
print(parts[0])
print("\t", part_tokens)
print("\t", bm)

Summarize in a function

In [ ]:
#| export 
def detect_columns(query: str, columns: List[str], growth_words: set[str], stopwords: set[str]) -> List[str]:
    """
    1. tokenize the columns that I want to predict, remove or add metadata relevant to the prediction
    2. Divide the query in subqueries or parts (I may want multiple facts)
    3. tokenize each subquery/part
    4. For each subquery, find the best match with my set of columns/financial_facts
    """
    q = query.lower()
    # split into parts by commas or the word 'and' (word-boundary)
    parts = [p.strip() for p in re.split(r',|\band\b', q) if p.strip()]
    
    # preprocess columns into token sets and growth-flag
    cols_meta = preprocess_columns(columns, stopwords)

    results = []
    for part in parts:
        part_tokens = set(t for t in _tokens(part) if t not in stopwords)
        if not part_tokens:
            continue

        # detect if user asked for growth/percentage in this part
        wants_growth = any(k in part for k in growth_words)

        # Prefer columns that match the growth intent; but if none produce positive score, allow the other group.
        # try matching within the requested growth/non-growth group
        primary_group = best_match(part_tokens, cols_meta, wants_growth)

        # If the best_match dictionary is not empty and the score is greater than 0 
        if primary_group[0] and (primary_group[1] > 0):
            # Then I will save the name of that column with the best match
            chosen = primary_group[0]['name']
        else:
            # fallback: try the opposite group
            secondary_group = best_match(part_tokens, cols_meta, not wants_growth)
            if secondary_group[0] and secondary_group[1] > 0:
                chosen = secondary_group[0]['name']
            
            else:
                # no confident match in either group -> try any column by best score (avoid choosing on 'total' only)
                # Look in the parts withouth passing the STOPWORDS filter
                best_overall = None
                best_score = 0
                for meta in cols_meta:
                    score = len(part_tokens & meta['tokens'])
                    if meta['name'].lower() in part:
                        score += 1
                    if score > best_score:
                        best_overall = meta
                        best_score = score
                # accept only if there's at least one token matched
                chosen = best_overall['name'] if best_overall and best_score > 0 else None

        if chosen and chosen not in results:
            results.append(chosen)

    return results


In [ ]:
# --- Examples from your message ---
q1 = "Give me the flow "
q2 = "Give me the revenue and the Total Liabilities"

print(q1, "\n\t->", detect_columns(q1, COLUMNS, GROWTH_KEYWORDS, STOPWORDS))
print(q2, "\n\t->", detect_columns(q2, COLUMNS, GROWTH_KEYWORDS, STOPWORDS))

Let's include this function as a method in the class

In [ ]:
#| export

@patch
def parse_financial_facts(self: financial_chatbot):
    
    assert isinstance(self._current_query, str)
    query = self._current_query

    columns = self._available_fin_facts
    growth_words = self._growth_keywords
    stopwords = self._stopwords
    
    found = detect_columns(query, columns, growth_words, stopwords)
    if len(found)>0:
        self._last_fin_facts = self._current_fin_facts
        self._current_fin_facts = found

    return found


In [ ]:
chatbot._current_query = "Print the Net results, Total assets and percentage assets"
chatbot.parse_financial_facts()

---

### Loop To ask about missing info

In [ ]:
#| export
@patch
def query_for_missing_info(self: financial_chatbot, info_type: str, prompt_message: str):
    counter = 0
    while True:
        if info_type == 'company' and len(self._current_companies) == 0:
            print(prompt_message)
            print(f"Examples {self._available_companies[:3]}")
            self.get_response()
            self.parse_companies()

        elif info_type == 'action' and len(self._current_action) == 0:

            if len(self.action_for)==0:
                self.initiate_actions_dictionary()

            print(prompt_message)
            self.get_response()
            self.parse_action()

        elif info_type == 'fact' and len(self._current_fin_facts) == 0:

            print(prompt_message)
            self.get_response()
            self.parse_financial_facts()
        else:
            break

        # Avoid too many attempts
        counter += 1
        if counter > 3:
            # print("Too many attempts. Exiting.")
            raise SystemExit("Too many attempts. Exiting.")
            # break


#### Tests

In [ ]:
cb = financial_chatbot(file_path=file_path)
cb.initiate_actions_dictionary()


Test that the loop stops the program:

In [ ]:
# The user inputs something not valid
__builtins__.input = lambda prompt="": "Fake answer"

try:
    print("Starting...")
    cb.query_for_missing_info(info_type="company", prompt_message="Please specify at least one company in your query: ")
    print("This won't run unless SystemExit is caught")
    
except SystemExit as e:
    print(f"Caught SystemExit: {e}")




Test with correct user prompts

In [ ]:
__builtins__.input = lambda prompt="": "Apple and Microsoft"
cb.query_for_missing_info(info_type="company", prompt_message="Please specify at least one company in your query: ")

__builtins__.input = lambda prompt="": "print"
cb.query_for_missing_info(info_type="action", prompt_message="Would you like me to print or plot the data? ")

__builtins__.input = lambda prompt="": "Net income and Total assets growth"
cb.query_for_missing_info(info_type="fact", prompt_message="What financial fact are you interested in?")


Method to print the saved information

In [ ]:
#| export
@patch
def _print_parsed_information(self: financial_chatbot):
    print(f"Selected companies: {self._current_companies}")
    print(f"Requested actions: {self._current_action}")
    print(f"Requested actions: {self._current_fin_facts}")

Results

In [ ]:
cb._print_parsed_information()

# Methods to filter and print/plot the queried data

Method to filter the data with the information from the queries

In [ ]:
#| export
@patch
def _get_queried_data(self: financial_chatbot, last_n_years: int = 3):
    """ 
    Once I have parsed all the information in my query, I extract it from the data
    To shorten the print, you can indicate how many years should be printed.
    """

    facts = self._current_fin_facts
    filter_company = self._data["Company"].isin(self._current_companies)
    cols = ["Company","Year"] + (facts if isinstance(facts, list) else [facts])

    filtered_Data = self._data[filter_company].sort_values(
        ['Company', 'Year'],
        ascending=[True, False]
        ).groupby('Company').head(last_n_years)[cols]

    return facts, filter_company, cols, filtered_Data


For plotting:

In [ ]:
#| export

@patch
def print_queried_data(self: financial_chatbot, last_n_years: int = 2):
    """ 
    It will print the filtered dataframe with the selected:
    Companies, Financial facts and the last 3 years to make it shorter
    """
    
    _, _, _, filtered_Data = self._get_queried_data(last_n_years)

    print(filtered_Data)


In [ ]:
cb.print_queried_data(1)

For plotting:

In [ ]:
#| export
import matplotlib.gridspec as gridspec
import math

In [ ]:
#| export
@patch
def plot_queried_data(self: financial_chatbot, last_n_years:int = 3):
    
    facts, _, _, filtered_Data = self._get_queried_data(last_n_years)

    m, n = math.ceil(len(facts)/2), 2 # Number of rows and columns

    # Create a grid
    fig = plt.figure(figsize=(4*n, 3*m))
    gs = gridspec.GridSpec(m, n, figure=fig)

    axs=[]
    for i, fact in enumerate(facts):
        ax = fig.add_subplot(gs[i])
        sns.barplot(data=filtered_Data, x="Year", y=fact, hue="Company", ax=ax)
        axs.append(ax)
    plt.tight_layout()
    plt.show()


In [ ]:
cb.plot_queried_data(3)

I can overwrite the ``initiate_actions_dictionary`` method to use the new methods for print and plot that I have just defined (In a final setup I could just order everything in a class withouth patches)

In [ ]:
#| export
@patch
def initiate_actions_dictionary(self: financial_chatbot):
    # Map print and plot keywords to their respective functions
    associate_action(self._keywords_print, self.print_queried_data, self.action_for)
    associate_action(self._keywords_plot, self.plot_queried_data, self.action_for)



In [ ]:
cb.initiate_actions_dictionary()


In [ ]:
cb.action_for["print"](1)

In [ ]:
cb.action_for["print"](2)
cb.action_for["plot"](2)

# Test logic

In [ ]:
test = True

cb = financial_chatbot(file_path=file_path)

# Initiate some variables
cb.initiate_actions_dictionary()


# First message 
cb.print_intro()

# First user query
if test: __builtins__.input = lambda prompt="": "Print and plot the Net income, Total assets and Cash Flow for Apple and Microsoft"
cb.get_response(message="How can I help you?")

# parse query for company
cb.parse_companies()

# parse query for action   
cb.parse_action()

# Parse Financial facts
cb.parse_financial_facts()


# Ask for missing information
cb.query_for_missing_info(info_type="company", prompt_message="Please specify at least one company in your query: ")
cb.query_for_missing_info(info_type="action", prompt_message="Would you like me to print or plot the data? ")
cb.query_for_missing_info(info_type="fact", prompt_message="What financial fact are you interested in?")

print("Summary:")
cb._print_parsed_information()

# At this point if I don't have all the needed information, the chatbot would have stopped
# Execute the action on the given data
print()
print(80*"-")
for action_i in cb._current_action:
    # print(action_i)
    cb.action_for[action_i]()
    print()
    print(80*"-")




Wrap everything in a method

In [ ]:
#| export
@patch
def _chatbot_iteration(self: financial_chatbot):
    """One loop of the chatbot"""

    # First user query
    self.get_response(message=" > How can I help you? \n > (Type 'exit' to end the chat) ")

    # parse query for company
    self.parse_companies()
    # parse query for action   
    self.parse_action()
    # Parse Financial facts
    self.parse_financial_facts()

    # Ask for missing information
    self.query_for_missing_info(info_type="company", prompt_message="Please specify at least one company in your query: ")
    self.query_for_missing_info(info_type="action", prompt_message="Would you like me to print or plot the data? ")
    self.query_for_missing_info(info_type="fact", prompt_message="What financial fact are you interested in?")

    print("Summary:")
    self._print_parsed_information()

    # At this point if I don't have all the needed information, the chatbot would have stopped
    # Execute the action on the given data
    print()
    print(80*"-")
    for action_i in self._current_action:
        # print(action_i)
        self.action_for[action_i]()
        print()
        print(80*"-")


Try how the main would look like:

In [ ]:
# Correct user input:
__builtins__.input = lambda prompt="": "Print the growth of Assets for Apple and Microsoft"

# Instantiate a chatbot
chatbot = financial_chatbot(file_path=file_path)

# Start the query loop
# Note: this should be a while True: 
for i in range(2):
    chatbot._chatbot_iteration()



# Export the main function

There is some additional code to check if the script is running from an interactive terminal or a tool like Github CI, so the run does not fail due to the program asking for user input.

In [ ]:
#| export

def is_interactive():
    return sys.stdin.isatty() and os.getenv("CI") != "true"

if __name__ == "__main__":
    if is_interactive():

        # Start chatbot
        print("Running in an interactive environment")

        base_dir = get_base_dir()
        path_to_data = base_dir.parent / "resources"
        file_name = "financial_data_chatbot.csv"
        file_path = path_to_data / file_name

        chatbot = financial_chatbot(file_path=file_path)

        while True:
            chatbot._chatbot_iteration()
            
    else:
        print("Skipping chatbot in non-interactive or CI environment.")

        

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()